# 1. This is the summary of Q1 & Q2

In [1]:
import sys
!{sys.executable} -m pip install wikipedia
import pandas as pd 
import wikipedia as wp
from bs4 import BeautifulSoup

html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df_postcode = pd.read_html(html, header = 0)[0]

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_postcode = df_postcode[df_postcode.Borough != 'Not assigned']

# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df_postcode = df_postcode.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.  
df_postcode['Neighborhood'].replace('Not assigned', df_postcode['Borough'], inplace = True)

# Import Library
import requests
import io

coordinates = pd.read_csv(io.StringIO(requests.get("http://cocl.us/Geospatial_data").content.decode('utf-8')))
coordinates.columns = ['Postal code', 'Latitude', 'Longitude']
df_postcode = pd.merge(coordinates, df_postcode, on = 'Postal code')
df_postcode = df_postcode[['Postal code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

# 2. This is the part of Q3

In [5]:
# Import Library
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from sklearn.cluster import KMeans

print('Libraries Imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

In [9]:
df_postcode.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


In [12]:
# Create Toronto Dataframe
df_toronto = df_postcode[df_postcode['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
boroughs = df_toronto['Borough'].unique().tolist()

In [44]:
# Visualisation
lat_toronto_mean = df_toronto['Latitude'].mean()
lon_toronto_mean = df_toronto['Longitude'].mean()

# Define Random Color
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))
    
# Create Map
map_toronto = folium.Map(location=[lat_toronto_mean, lon_toronto_mean], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = folium.Popup(label_text, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        tooltip = label_text,
        radius = 8,
        popup = label,
        color = borough_color[borough],
        fill = True,
        fill_color = borough_color[borough],
        fill_opacity = 0.8).add_to(map_toronto) 

map_toronto

Intuitively, there are four clusters in the map generated.